In [2]:
# Import modules

import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET
import requests
import pandas

In [3]:
# Open the file

xml_path = os.path.join('/home/clodaghboland/drugbankfile1.xml.gz')
with gzip.open(xml_path) as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

In [4]:
# Download data and add to dataframe

ns = '{http://www.drugbank.ca}'
rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['name'] = drug.findtext(ns + "name")
            row['type'] = drug.get('type')
            row['biologic_classification']=drug.findtext(ns + 'biologic_classification')
            row['groups'] = [group.text for group in
                drug.findall("{ns}groups/{ns}group".format(ns = ns))]
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            ref_text = protein.findtext("{ns}references[@format='textile']".format(ns=ns))
            rows.append(row)

df = pandas.DataFrame.from_dict(rows)


In [6]:
# Create a list of myotoxins using their DrugBank ID's

myotoxins = ['DB05812','DB00459','DB00518','DB00041','DB09026','DB00009','DB00513','DB01223','DB01118','DB00543','DB00182','DB00681','DB01238',
             'DB06216','DB01072','DB00289','DB01076','DB11945','DB00181','DB00865','DB09128','DB01156','DB00796','DB06016','DB00535','DB00439',
             'DB00215','DB01211','DB00363','DB09065','DB05239','DB01394','DB00531','DB00091','DB01406','DB00080','DB01264','DB01254','DB00705',
             'DB01189','DB06700','DB12161','DB06701','DB01576','DB00271','DB00900','DB00343','DB00843','DB09272','DB00494','DB00876','DB00530',
             'DB00199','DB01175','DB06414','DB01590','DB00973','DB00927','DB04854','DB00949','DB01039','DB13873','DB00196','DB01095','DB00176',
             'DB01319','DB00996','DB01004','DB01241','DB00986','DB00502','DB01181','DB04946','DB00619','DB00224','DB00105','DB01249','DB01362',
             'DB08947','DB09156','DB09133','DB09134','DB01029','DB00982','DB01167','DB01026','DB00709','DB00555','DB12070','DB01137','DB01255',
             'DB08827','DB00678','DB00227','DB00408','DB08815','DB04835','DB01577','DB00422','DB01233','DB00834','DB04896','DB00370','DB00295',
             'DB01149','DB01280','DB00220','DB00238','DB00627','DB09035','DB01165','DB00334','DB00275','DB00526','DB01267','DB00213','DB00715',
             'DB00022','DB09323','DB01100','DB08860','DB01263','DB00413','DB00175','DB00230','DB00818','DB01224','DB00468','DB01129','DB06817',
             'DB00015','DB01201','DB00734','DB00503','DB01098','DB05271','DB01232','DB01104','DB06290','DB00641','DB06688','DB00877','DB09143',
             'DB00398','DB00202','DB00795','DB01268','DB00966','DB06287','DB00031','DB00300','DB00857','DB04844','DB00277','DB00932','DB00323',
             'DB06212','DB05109','DB08911','DB06824','DB00209','DB00013','DB00177','DB00067','DB00285','DB00661','DB00582','DB06283','DB00495',
             'DB00246','DB00909','DB00321','DB01142','DB00472','DB00623','DB00344','DB00829','DB01595','DB01544','DB01393','DB01452','DB00907',
             'DB01075','DB00366','DB00780','DB00850','DB01069','DB00477','DB00420','DB00831','DB00186','DB00897','DB01437','DB00636','DB09064',
             'DB00333','DB00993','DB01159','DB04570','DB01580','DB00316','DB00859','DB00738','DB03575','DB00252','DB00397','DB00908','DB15954',
             'DB00871','DB01258','DB00625','DB00529','DB00945','DB00537','DB00069','DB01053','DB01265','DB01042','DB06288','DB00572','DB00928',
             'DB00443','DB00201','DB02329','DB01563','DB00608','DB00987','DB00514','DB00586','DB01551','DB13694','DB01454','DB00668','DB00574',
             'DB13042','DB13751','DB00741','DB00557','DB00034','DB09487','DB00753','DB00951','DB01221','DB00598','DB00431','DB00150','DB04829',
             'DB00454','DB00371','DB00704','DB01059','DB01579','DB00914','DB01620','DB00830','DB01174','DB00191','DB00960','DB01132','DB00635',
             'DB00852','DB00867','DB00936','DB01236','DB01105','DB01208','DB06718','DB00864','DB00231','DB00313']
myotoxin_df=df.loc[df['drugbank_id'].isin(myotoxins)] #extract information on myotoxins only and add to a new dataframe

In [23]:
# Export data to an excel file

myotoxin_df.to_excel('myotoxic_data.xlsx')